### Concept: JSON File

In [1]:
import json
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)
    #print(json_data)

## Todo#1
1. Thử un-comment bằng cách để dấu nháy chuột ở dòng print(json_data) và nhấn Ctrl+/. Sau đó, chạy lại.
2. Cấu trúc của json_data tương tự dictionary: thử gọi biến SENDER_EMAIL trong file json và print => Task này sẽ giúp bạn tìm hiểu thêm về cấu trúc của 1 file JSON.

In [2]:
sender=json_data['SENDER_EMAIL']
print(sender)

ha.ngoc1476@gmail.com


### Concept: Environment Variables

In [3]:
import os
os.environ['SENDER_EMAIL'] = json_data['SENDER_EMAIL'] ## INPUT: Your Email
os.environ['PWD_EMAIL'] = json_data['PWD_EMAIL'] ## INPUT: Your App Password
os.environ['WEATHER_API_KEY'] = json_data['WEATHER_API_KEY'] ## INPUT: OpenWeather API Key
print('DONE! Get Env. Variable')

DONE! Get Env. Variable


In [4]:
print(os.environ['SENDER_EMAIL'])

ha.ngoc1476@gmail.com


 ### Import Modules

In [5]:
import email, smtplib, ssl # to automate email
import email as mail
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

import requests, json # to pull API, and work with json
import datetime as dt # to work with date, time
from bs4 import BeautifulSoup # to work with web scrapping (HTML)
import pandas as pd # to work with tables (DataFrames) data
from IPython.core.display import HTML # to display HTML in the notebook

### Email Bot

In [7]:
subject = 'Test Email' #INPUT1: Subject of the Email
receiver_email = 'ha.ngoc1476@gmail.com'
sender_email = os.environ['SENDER_EMAIL']
password = os.environ['PWD_EMAIL'] 

# (1) Create the email head (sender, receiver, and subject)
email = MIMEMultipart()
email["From"] = sender_email
email["To"] = receiver_email 
email["Subject"] = subject

# (2) Create Body part
# We use html, you can convert word to html: https://wordtohtml.net/
html1 = """
    <html>
    <body>
    <p>Hello,</p>
    <p>How are you?</p>
    <p>Click <a href="https://www.google.com/">here</a> to download.</p>
    </body>
    </html>
    """
html2 = """
<html>
Email sent at <b>{}</b><br>
</html>
""".format(dt.datetime.now().isoformat())

text3 = '--- End ----'

# Combine parts
part1 = MIMEText(html1, 'html')
part2 = MIMEText(html2, 'html')
part3 = MIMEText(text3, 'plain')

email.attach(part1)
email.attach(part2)
email.attach(part3)

# (3) Create SMTP session for sending the mail
session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
session.starttls() #enable security
session.login(sender_email, password) #login with mail_id and password
text = email.as_string()
session.sendmail(sender_email, receiver_email, text)
session.quit()

print(f'DONE! Mail Sent to {receiver_email}')

DONE! Mail Sent to ha.ngoc1476@gmail.com


## TODO#2
1. Từ Email Bot Code viết function send_email:
inputs: subject, receiver_email, part1, part2, part3
output: thực hiện việc gửi email và in ra kết quả
2. Kết hợp Loops (For ... in) với function để gửi cùng nội dung email đến 1 list user: email_list = ['abc@gmail.com', 'xyz@gmail.com'] (thay đổi email bằng các email thật của bạn để test)

In [20]:
# Write function to send the email
def send_email(receiver_email, subject, part1, part2, part3):
    # (1) Create the email head
    sender_email = os.environ['SENDER_EMAIL']
    password = os.environ['PWD_EMAIL']
    
    email = MIMEMultipart()
    email['From'] = sender_email
    email['To'] = receiver_email
    email['Subject'] = subject
    #(2) Attach mail
    email.attach(part1)
    email.attach(part2)
    email.attach(part3)
    # (3) Create SMTP session for sending the mail
    try:
        session = smtplib.SMTP('smtp.gmail.com', 587) #use gmail with port
        session.starttls() #enable security
        session.login(sender_email, password) #login with mail_id and password
        text = email.as_string()
        session.sendmail(sender_email, receiver_email, text)
        print('Done!. Mail Sent to {}'.format(receiver_email))
    except:
        print('Email delivery failure')
send_email(receiver_email, subject, part1, part2, part3)

Done!. Mail Sent to ha.ngoc1476@gmail.com


In [22]:
#use function above to send the email
email_list = ['ngocha.nt1476@gmail.com', 'ha.ngoc1476@gmail.com']
for receiver_email in email_list:
    send_email(receiver_email, subject, part1, part2, part3)

Done!. Mail Sent to ngocha.nt1476@gmail.com
Done!. Mail Sent to ha.ngoc1476@gmail.com


### Call API
. API (Application Programming Interface): Là cách thức và phương tiện để 2 applications nói chuyện với nhau. Ở đây là giữa Python program trên Colab của bạn và các Web application khác.

. Đây là concept nền tảng trong việc Automation
API nhận input là JSON file (có khi không nhận) và trả lại output là JSON file => Request/Call API

. Đây là một cách phổ biến đế gửi và nhận data

Đoạn code bên dưới:

. Lấy input là API_KEY, country, ta request API của Open Weather

. Được trả lại file kết quả dưới dạng JSON

. Trích xuất 1 vài thông tin, và lưu ra file text

In [23]:
# base URL
BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
# City Name 
CITY = "London"
# API key 
API_KEY = os.environ['WEATHER_API_KEY']
# upadting the URL
URL = BASE_URL + "q=" + CITY + "&appid=" + API_KEY
# HTTP request
response = requests.get(URL)
# checking the status code of the request
if response.status_code == 200:
   # getting data in the json format
   data = response.json()
   # getting the main dict block
   main = data['main']
   # getting temperature
   temperature = main['temp']
   # getting the humidity
   humidity = main['humidity']
   # getting the pressure
   pressure = main['pressure']
   # weather report
   report = data['weather']
   f = open('weather.txt','w') ## write to text file
   print(f'{CITY:-^30}', file=f)
   print(f'Temperature: {temperature}', file=f)
   print(f'Humidity: {humidity}', file=f)
   print(f'Pressure: {pressure}', file=f)
   print(f"Weather Report: {report[0]['description']}", file=f)
   f.close()
   print('DONE! Output in weather.txt file')
else:
   # showing the error message
   print('Error in the HTTP request')
   print(response)

DONE! Output in weather.txt file


In [24]:
print(open("weather.txt", "r").read())

------------London------------
Temperature: 281.65
Humidity: 81
Pressure: 1005
Weather Report: overcast clouds



In [25]:
# Khám phá file JSON
data

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 281.65,
  'feels_like': 280.54,
  'temp_min': 280.93,
  'temp_max': 282.59,
  'pressure': 1005,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 2.06, 'deg': 150},
 'clouds': {'all': 100},
 'dt': 1621046042,
 'sys': {'type': 1,
  'id': 1414,
  'country': 'GB',
  'sunrise': 1621051712,
  'sunset': 1621107921},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

In [26]:
data['name']

'London'

In [27]:
data['coord']['lat']

51.5085

## TODO#3

#### 1. Trên file json kết quả:
. Lấy thông tin trạng thái thời tiết (weather) chính (main) ở London

. Lấy lat & lon (kinh độ và vĩ độ của London)

In [28]:
#getting main weather
main_weather=data['weather'][0]['main']
#getting lat
lat = data['coord']['lat']
#getting lat
lon = data['coord']['lon']
city=data['name']
#print the main weather, lat, lon
print(f'The main weather of {city} is {main_weather}\nLongtitude is {lon} and Latitude is {lat}')

The main weather of London is Clouds
Longtitude is -0.1257 and Latitude is 51.5085


#### 2. Viết function get_weather_data
. inputs: city

. outputs: đoạn text thông tin thời tiết

In [61]:
def get_weather_data(city):
    # base URL
    BASE_URL = "https://api.openweathermap.org/data/2.5/weather?"
    # API key 
    API_KEY = os.environ['WEATHER_API_KEY']
    # upadting the URL
    URL = BASE_URL + "q=" + city + "&appid=" + API_KEY
    # HTTP request
    response = requests.get(URL)
    # checking the status code of the request
    if response.status_code == 200:
        # getting data in the json format
        data = response.json()
        # getting the main dict block
        main = data['main']
        # getting temperature
        temperature = main['temp']
        # getting the humidity
        humidity = main['humidity']
        # getting the pressure
        pressure = main['pressure']
        # weather report
        report = data['weather']
        main_report = report[0]['description']
        
        # test the weather text
        output = f'The main weather of {city} is {main_report}.\nThe temperature is {temperature}, humidity is {humidity} and pressure is {pressure}.'
    else:
        # showing the error message
        output = 'Error in the HTTP request'
    return(output)
text=get_weather_data('Hanoi')
print(text)

The main weather of Hanoi is broken clouds.
The temperature is 304.15, humidity is 74 and pressure is 1007.


#### 3. Dùng function send_email in TODO #1, thay phần 3 trong nội dùng email bằng thông tin thời tiết của London gửi cho 1 địa chỉ email của bạn

In [63]:
#send mail to 
receiver_email ='ha.ngoc1476@gmail.com'
subject = 'Hanoi Weather'
city='Hanoi'
# get weather data
text3 = get_weather_data(city)
part3 = MIMEText(text3, 'plain')
#use function send_mail
send_email(receiver_email, subject, part1, part2, part3)

Done!. Mail Sent to ha.ngoc1476@gmail.com


### Concept: Web Scrapping

In [47]:
response = requests.get(
	url="https://en.wikipedia.org/wiki/COVID-19",
  )
print("API Status Code: "+ str(response.status_code))
soup = BeautifulSoup(response.content, 'html.parser')

title = soup.find(id="firstHeading")
print(title.string)

API Status Code: 200
COVID-19


In [50]:
# Get all the links
p = soup.find(id="bodyContent").find_all("p")

In [53]:
from IPython.core.display import HTML
HTML(str(p[3]))

## TODO#4
1. Dùng function send_email in Excercise 1.1, thay phần 3 trong nội dùng email bằng thông tin COVID-19 từ Wikipedia gửi cho 1 địa chỉ email của bạn

In [44]:
text3 = str(p[3])  #change part3 into the content of Covid-19
part3 = MIMEText(text3, 'html')
# send mail
subject = 'Covid information' #INPUT1: Subject of the Email
receiver_email = 'ha.ngoc1476@gmail.com' #INPUT2: email address to receive the email

send_email(receiver_email, subject, part1, part2, part3)

Done!. Mail Sent to ha.ngoc1476@gmail.com


### Concept CSV
. CSV là dạng data có các cột được ngăn cách bằng dấu phẩy(Comma-separated value)

. Đây là dạng data (bảng) cực kỳ phổ biến trong Data Analytics & Data Science

. Có thể đọc, ghi, và xử lý bằng pandas

In [229]:
csv = """email,contents,city
abc@gmail.com,News,Hanoi
xyz@gmail.com,Weather,London
"""
f = open('email_list.csv','w') ## write to text file
f.write(csv)
f.close()

In [230]:
import pandas as pd
email_list = pd.read_csv('email_list.csv')

In [231]:
email_list

,email,contents,city
0,abc@gmail.com,News,Hanoi
1,xyz@gmail.com,Weather,London


## TODO#5
1. Add các địa chỉ email có ý nghĩa, và thêm dòng vào file csv (lưu ý không có dấu cách)
2. Dùng code để gửi email cho một list từ csv:
. Nếu email có contents = 'News' => Gửi thông tin COVID19
. Nếu email có contents = 'Weather' => Gửi thông tin thời tiết theo thành phố trong cột city (lưu ý check các city được hỗ trợ trong API)

In [227]:
def email_list_csv(email_list):
    csv = """email,contents,city
    ha.ngoc1476@gmail.com,News,London
    ngocha.nt1476@gmail.com,Weather,Hanoi
    """
    f = open('email_list.csv','w') ## write to text file
    f.write(csv)
    f.close()

In [244]:
email_list = pd.read_csv('email_list.csv')
for i in range(len(email_list)):
    print('Email #{}'.format(i))
    receiver = email_list['email'][i]
    content = email_list['contents'][i]
    city = email_list['city'][i]

    if content == 'News':
        print('COVID To: {}'.format(receiver))
        subject = 'Covid information'
        part3 = MIMEText(str(p[3]), 'html')
    elif content == 'Weather':
        print("Weather To: {}".format(receiver))
        subject = 'Hanoi Weather'
        part3 = MIMEText(text3, 'plain')
    else:
        print('Invalid Contents')
send_email(receiver_email, subject, part1, part2, part3)
print('DONE!')

Email #0
COVID To:     ha.ngoc1476@gmail.com
Email #1
Weather To:     ngocha.nt1476@gmail.com
Done!. Mail Sent to ha.ngoc1476@gmail.com
DONE!
